![copyright](copyright.png "Copyright (c) 2017, Bas Hoonhout")

# 3. Numerics

This notebook introduces you to some of the detailed numerics of AeoLiS.
Along with this notebook a series of AeoLiS model schematizations of the Sand Motor is provided that you can use to investigate the effect of some of the more advanced options in AeoLiS.
Note that these model schematizations are highly optimized for speed and only have an educational purpose.

Execute cells by selecting the cell and press **Shift-Enter**.

### Helper functions

Execute the next cell with **Shift-Enter** to enable the helper functions.
Adapt the functions if appropriate.

In [ ]:
%matplotlib inline

from plot_tools import *

## 3.1 Timescales

In AeoLiS timescales are used to parameterize two decay processes that are relevant to coastal aeolian sediment transport. The first timescale is the adaptation timescale `T` in the advection equation that determines the rate of pickup in downwind direction when transport is not saturated. The second timescale is the drying timescale `Tdry` in the soil moisture model that determines the drying of the soil after inundation.

Both timescales influence the source area of aeolian sediment in terms of location and composition. What happens to the source area when you:
- Increase the adaptation timescale `T` with a factor 10?
- Increase the drying timescale `Tdry` with a factor 3?

### Questions

- Can the adaptation timescale `T` be smaller than the model time step? Why?
- What process is the drying timescale `Tdry` essentially representing?
- The adaptation timescale `T` is related to the critical fetch concept. How?

In [ ]:
ncfile = '../models/coarse/sandmotor_10year_sorting1_tide1_waves1.nc'
plot_bathymetry(ncfile)
plot_bathymetry(ncfile, change=True)
plot_coverage(ncfile)

## 3.2 Drag partitioning

The influence of large non-erodible elements (e.g. shells) on a sandy bed extends beyond the area that is covered by a particular element. Shadow effects in the lee of a non-erodible element cause the local wind shear to be lowered, resulting in a decrease in aeolian sediment transport capacity. This shadow effect is referred to as drag partitioning and included in AeoLiS through the Raupach (1993) model.

Raupach (1993) defines the parameters `sigma`, `beta` and `m` for parameterizing the drag partitioning in terms of a roughness density `lambda`. Mathematically `sigma` and `beta` only appear in the implementation of AeoLiS as a ratio and therefore essentially represent a single parameter. `m` is either 0.5 or 1.0 depending on whether you are dealing with a lab or field situation.

### Questions

You can use the function in the next cell to read variable values from the AeoLiS result file to answer the following questions:

- What are the dimensions of the model state variable `mass` that describes the sediment mass in the bed.
- The roughness density `lambda` and the `D50` are not a state variables in AeoLiS and can therefore not be read directly. Can you construct the roughness density `lambda` and `D50` from the AeoLiS result file?

In [ ]:
def read_variable(ncfile, var=None):
    '''Read arbitrary variable value from a netCDF file
    
    If no variable name is given, all available variables are listed.
    
    Parameters
    ----------
    ncfile : str
      Path to AeoLiS result file
    var : str or list, optional
      Name of variable or list with variable names
      
    Results
    -------
    val : np.ndarray or list
      Array with variable values or list with variable names
      
    '''
    
    with netCDF4.Dataset(ncfile, 'r') as ds:
        if var is None:
            return list(ds.variables.keys())
        elif type(var) is str:
            return ds.variables[var][...]
        elif type(var) is list:
            return tuple([ds.variables[v][...] for v in var])
        else:
            raise TypeError('Unknown variable type [%s]' % type(var))

In [ ]:
print(', '.join(read_variable('../models/coarse/sandmotor_10year_sorting1_tide1_waves1.nc')))

In [ ]:
x, zb = read_variable('../models/coarse/sandmotor_10year_sorting1_tide1_waves1.nc', ['x','zb'])

fig, ax = plt.subplots(figsize=(10,4))
ax.plot(x[0,:], zb[-1,:,:].T, '-k');
ax.set_xlabel('cross-shore [m]')
ax.set_ylabel('bed level [m]')
ax.set_xlim([-750, 750])
ax.set_ylim([-10, 15])
ax.grid()

## 3.3 Bed interaction

For the numerical implementation of multi-fraction sediment transport a distribution of grain size fraction for the erosion and deposition is needed. For hydrodynamic models the grain size distribution in the local bed cell is typically used as distribution. The advantage of such approach is that it is fairly trivial to implement. The disadvantage of this approach is that fine grains cannot easily pass a patch of coarse grains or vice versa, which hampers the development of spatial variations in bed composition. Although this disadvantage might be acceptable for hydrodynamic processes, it poses significant limitations for modeling aeolian sediment transport processes.

In AeoLiS the grain size distribution for sediment erosion and deposition depends on both the grain size distribution in the bed and in the air. The bed interaction factor weighs both distribution to obtain the grain size distribution for erosion and deposition. A bed interaction factor of zero means that, in case of saturated transport, sediment can pass without interaction with the bed. A value of one means that the continuation of transport depends entirely on the grain size distribution in the bed.

### Questions

- What is the default value of the bed interaction factor?
- What is the effect on the bed composition at the Sand Motor when the bed interaction factor is set to zero?
- What is the effect on the bed composition at the Sand Motor when the bed interaction factor is set to one?

In [ ]:
plot_coverage('../models/coarse/sandmotor_10year_sorting1_tide1_waves1.nc')

## 3.4 Subtidal bathymetry / model coupling

In AeoLiS hydrodynamics are only used to determine beach width and wave reworking of the bed.
In case of the Sand Motor, hydrodynamics also affect the subtidal bathymetry signifcantly due to current- and wave-driven sediment transport that diffuses the bulge shaped nourishment along the original coastline.
The subtidal morphological change is not represented in AeoLiS.
However, through AeoLiS' Basic Model Interface (BMI) it is possible to interact with the model while running.
BMI can be used to couple AeoLiS to other models (e.g. XBeach or Delft3D FM) or to data.
A Python plugin `callback.py` is provided to update AeoLiS with measured bathymetries.
The plugin is designed for the original Sand Motor model and needs to be adapted to work with the speed-optimized version of the Sand Motor model.

### Questions

- Can you enable the plugin for the speed-optimized Sand Motor model (hint: look for *callback* in the documentation).
- The provided Python plugin uses the fine gridded bathymetry for updating the AeoLiS bathymetry, which is not compatible with the speed-optimized version of the Sand Motor model. Can you adapt the plugin to make it work with the coarse grid?
- Can you plot the bed level change computed by AeoLiS? Is the bed level change computed by AeoLiS equal to the bed level change you imposed from the measurements?

In [ ]:
plot_bathymetry('../models/coarse/sandmotor_10year_sorting1_tide1_waves1.nc')